# Occupancy prediction demo

This notebook demonstrates how to **load a fitted occupancy model** and **predict occupancy for a new site**
without relying on the `mmocc` package or pre-computed features. It:

- installs required packages (works locally or on Google Colab),
- downloads our fitted models,
- downloads a public camera‑trap image, and
- computes an image embedding on‑the‑fly to produce an occupancy probability.

In [ ]:
species_id = "16ec4010-f175-4de7-8a99-85aadec3963b"  # western gray squirrel, see https://www.kaggle.com/models/google/speciesnet?select=taxonomy_release.txt for other species IDs

In [ ]:
# Install required packages (local + Colab friendly)
import importlib.util
import subprocess
import sys


def ensure_pkg(import_name: str, pip_name: str) -> None:
    if importlib.util.find_spec(import_name) is None:
        print(f"Installing {pip_name}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pip_name])


packages = [
    ("numpy", "numpy"),
    ("pandas", "pandas"),
    ("PIL", "Pillow"),
    ("matplotlib", "matplotlib"),
    ("sklearn", "scikit-learn"),
    ("torch", "torch"),
    ("torchvision", "torchvision"),
]

for import_name, pip_name in packages:
    ensure_pkg(import_name, pip_name)

In [ ]:
# Imports
import os
import pickle
import tarfile
import urllib.request

import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import torch
from torchvision import transforms

In [ ]:
# Download cache archive if not already present
cache_url = 'https://data.csail.mit.edu/mmocc/mmocc_cache.tar.gz'
cache_filename = 'mmocc_cache.tar.gz'

if not os.path.exists(cache_filename):
    print(f'Downloading {cache_filename}...')
    urllib.request.urlretrieve(cache_url, cache_filename)
    print('Download complete.')

In [ ]:
# Download a public camera‑trap image (Caltech Camera Traps)
image_url = (
    "https://lilawildlife.blob.core.windows.net/lila-wildlife/"
    "caltech-unzipped/cct_images/5968c0f9-23d2-11e8-a6a3-ec086b02610b.jpg"
)
image_path = "demo_camera_trap.jpg"
urllib.request.urlretrieve(image_url, image_path)
img = Image.open(image_path).convert("RGB")
plt.imshow(img)
plt.axis("off")
plt.title("Downloaded camera‑trap image")
plt.show()

In [ ]:
# Select an image‑only model (avoids cached satellite/covariate features)
fit_filename = (
    f'{species_id}_modalities_image_image_dinov2_vitb14_sat_None.pkl'
)

with tarfile.open(cache_filename, 'r:gz') as tar:
    # Find the fit_results file inside the archive
    candidates = [
        member for member in tar.getmembers()
        if member.name.endswith(f'/fit_results/{fit_filename}') or member.name.endswith(fit_filename)
    ]
    if not candidates:
        raise FileNotFoundError(f'Could not find {fit_filename} inside {cache_filename}')
    if len(candidates) > 1:
        print('Warning: multiple matches found; using the first one:', candidates[0].name)
    member = candidates[0]
    file_obj = tar.extractfile(member)
    if file_obj is None:
        raise FileNotFoundError(f'Failed to extract {member.name} from {cache_filename}')
    fit_results = pickle.load(file_obj)

display_name = (
    fit_results.get('common_name')
    or fit_results.get('scientific_name')
    or fit_results.get('taxon_id')
)
print('Taxon:', display_name)
print('Modalities:', fit_results['modalities'])
print('Image backbone:', fit_results.get('image_backbone'))

In [ ]:
# Compute an image embedding on‑the‑fly (no cached features)
backbone = torch.hub.load("facebookresearch/dinov2", "dinov2_vitb14")
backbone.eval()

device = "cuda" if torch.cuda.is_available() else "cpu"
backbone = backbone.to(device)

transform = transforms.Compose(
    [
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
        ),
    ]
)

with torch.no_grad():
    input_tensor = transform(img).unsqueeze(0).to(device)
    image_features = backbone(input_tensor).cpu().numpy().reshape(-1)

image_features.shape

In [ ]:
# Predict occupancy using the fitted model coefficients (point estimate)
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


modalities = fit_results["modalities"]
if modalities != ["image"]:
    raise ValueError("This demo only computes image features for an image-only model.")

scalers = fit_results["modalities_scaler"]
pcas = fit_results["modalities_pca"]
coefficients = fit_results.get("coefficients")
if coefficients is None:
    raise ValueError("Fit results missing coefficients needed for prediction.")
coefficients = np.asarray(coefficients)

features = {"image": image_features}

transformed = []
for modality in modalities:
    feats = features[modality].reshape(1, -1)
    reduced = pcas[modality].transform(scalers[modality].transform(feats))
    transformed.append(reduced)

combined = np.concatenate(transformed, axis=1)
logit = coefficients[0] + (combined @ coefficients[1:]).item()
prob = float(sigmoid(logit))

pd.DataFrame(
    [
        {
            "taxon": display_name,
            "modalities": ", ".join(modalities),
            "occupancy_prob": prob,
        }
    ]
)